# Reserve Probability Calculator

This app can be used to calculate the probabity of reserves given a maximum and minimum estimate. It is **important** to set the hydrocarbon type at the beginning of the problem. Care must be taken when inputting the initial and final rates to make sure they are in the correct units. The compatible units are as follows:
<br />
<br />
$$ Q_{oil} = \frac {bbl}{month} $$
<br />
$$ Q_{gas} = \frac {Mscf}{month} $$
<br />
Alternatively, you can input values directly from PHDWin. Inputting the values as they appear in the decline curve properties box in PHDWin will suffice. The app automatically converts De into the units required for the hyperbolic decline equation.

In [5]:
import numpy as np
import math as m
from scipy import stats
import pandas as pd
import matplotlib.pyplot as plt
from ipywidgets import BoundedFloatText,  interact,FloatSlider, Dropdown

def reserve(qi,qf,b,d):
    import numpy as np
    
    di = -np.log(1-d/100)/12
    
    Np = (qi**b)*(qi**(1-b)-qf**(1-b))/(di*(1-b))
    
    return Np

def dca(qi,qf,b,d):
    
    import numpy as np
    
    di = -np.log(1-d/100)/12 
    
    tf = ((qi/qf)**b-1)/(b*di)
    
    t = np.linspace(0,tf,100)
    q = np.zeros(len(t))
    Np = np.zeros(len(t))
    
    for i,j in enumerate(t):
        
        with np.errstate(divide='ignore'):
            
            
            q[i] = qi/((1+b*di*j)**(1/b)) 
    
    return (t,q)
    
    
def tdca(HC,qimax,qfmax,bmax,dmax,qimin,qfmin,bmin,dmin,qi50,qf50,b50,d50):
    
    tmax,qmax = dca(qimax,qfmax,bmax,dmax)
    tmin,qmin = dca(qimin,qfmin,bmin,dmin)
    t50,q50 = dca(qi50,qf50,b50,d50)
    
    Npmax = reserve(qimax,qfmax,bmax,dmax)
    Npmin = reserve(qimin,qfmin,bmin,dmin)
    Np50 = reserve(qi50,qf50,b50,d50)
    
#     fig, (ax1, ax2) = plt.subplots(1, 2)
#     plt.subplots_adjust(left=0, bottom=5, right=2, top=6, wspace=0.2, hspace=None)
#     plt.subplot(121)
#     plt.semilogy(tmax,qmax,tmin,qmin,t50,q50)
#     plt.gca().legend(('Max','Min','DCA Fitting'))
#     plt.xlabel('Months')
#     if HC == 'Oil':
        
#         plt.ylabel('HC production rate [bbl/month]')
#     else:
#         plt.ylabel('HC production rate [Mscf/month]')
        
#     plt.grid()
#     plt.show()
    
    
    
    if HC == 'Oil':

        qidiv = np.linspace(qimin,qimax,5)
        qtdiv = np.linspace(qfmin,qfmax,5)
        bdiv = np.linspace(bmin,bmax,10)
        ddiv = np.linspace(dmin,dmax,10)
        
        EUR = []
        
        for qi in qidiv:
            for qt in qtdiv:
                for b in bdiv:
                    for d in ddiv:
                        v = reserve(qi,qt,b,d)
                        EUR.append(v)
        EUR.sort()
        
        p10 = np.percentile(EUR,90)
        p50 = np.percentile(EUR,50)
        p90 = np.percentile(EUR,10)
        
        data = {'Reserve':[Np50/1000, p10/1000, p50/1000,p90/1000],'Units':['Mbbl','Mbbl','Mbbl','Mbbl']}
        df = pd.DataFrame(data, index =['DCA','P10', 'P50', 'P90'])
        
        
        Percentiles=stats.rankdata(EUR, "average")/len(EUR)
        EUR_cdf = np.asarray(EUR)
        reverse_EUR = EUR_cdf[::-1]
#         plt.subplot(122)
#         plt.plot(reverse_EUR/1000,Percentiles)
#         plt.xlabel('Reserve [Mbbl]')
#         plt.ylabel('Probability')
#         plt.grid()
#         plt.show()

        fig, (ax1, ax2) = plt.subplots(1, 2)
        plt.subplots_adjust(left=0, bottom=5, right=2, top=6, wspace=0.2, hspace=None)
        
        ax1.semilogy(tmax,qmax,'r--',tmin,qmin,'b--',t50,q50,'g-')
        ax1.grid()
        ax1.set(xlabel='Months', ylabel='Reserve [Mbbl]')
        ax1.legend(['Max','Min','DCA Fitting'])
        
        ax2.plot(reverse_EUR/1000,Percentiles)
        ax2.grid()
        ax2.set(xlabel='Reserve [Mbbl]', ylabel='Probability')
    
    
        
    if HC == 'Gas':

        qidiv = np.linspace(qimin,qimax,5)
        qtdiv = np.linspace(qfmin,qfmax,5)
        bdiv = np.linspace(bmin,bmax,10)
        ddiv = np.linspace(dmin,dmax,10)
        
        EUR = []
        
        for qi in qidiv:
            for qt in qtdiv:
                for b in bdiv:
                    for d in ddiv:
                        v = reserve(qi,qt,b,d)
                        EUR.append(v)
        EUR.sort()
        
        p10 = np.percentile(EUR,90)
        p50 = np.percentile(EUR,50)
        p90 = np.percentile(EUR,10)
        
        data = {'Reserve':[Np50/1000000, p10/1000000, p50/1000000,p90/1000000],'Units':['Bscf','Bscf','Bscf','Bscf']}
        df = pd.DataFrame(data, index =['DCA','P10', 'P50', 'P90'])  
        
        Percentiles=stats.rankdata(EUR, "average")/len(EUR)
        EUR_cdf = np.asarray(EUR)
        reverse_EUR = EUR_cdf[::-1]
#         plt.plot(reverse_EUR/1000,Percentiles)
#         plt.xlabel('Reserve [Bscf]')
#         plt.ylabel('Probability')
#         plt.grid()
#         plt.show()
        fig, (ax1, ax2) = plt.subplots(1, 2)
        plt.subplots_adjust(left=0, bottom=5, right=2, top=6, wspace=0.2, hspace=None)
        
        ax1.semilogy(tmax,qmax,'r--',tmin,qmin,'b--',t50,q50,'g-')
        ax1.grid()
        ax1.set(xlabel='Months', ylabel='Reserve [Bscf]')
        ax1.legend(['Max','Min','DCA Fitting'])
        
        ax2.plot(reverse_EUR/1000,Percentiles)
        ax2.grid()
        ax2.set(xlabel='Reserve [Bscf]', ylabel='Probability')
    
    return df



In [6]:
# #test values
# qimx=16935
# qtmx=104
# bmx=0.6
# dmx=40
# qimn=16935
# qtmn=104
# bmn=0.3
# dmn=80
# qifit=16935
# qtfit=104
# bfit=0.5
# dfit=50


# tdca('Oil',qimx,qtmx,bmx,dmx,qimn,qtmn,bmn,dmn,qifit,qtfit,bfit,dfit) 

In [7]:
Iqimax = BoundedFloatText(value=16935,min=0,max=1000000000,step=0.001,description='$Qi_{max}$',disabled=False)
Iqfmax = BoundedFloatText(value=100,min=0,max=1000000000,step=0.001,description='$Qt_{max}$',disabled=False)
Ibmax = BoundedFloatText(value=0.6,min=0.0001,max=1,step=0.0001,description='$b_{max}$',disabled=False)
Idmax = BoundedFloatText(value=77.38697,min=0,max=99.99999,step=0.0001,description='$De_{max}$',disabled=False)

Iqimin = BoundedFloatText(value=16935,min=0,max=1000000000,step=0.001,description='$Qi_{min}$',disabled=False)
Iqfmin = BoundedFloatText(value=100,min=0,max=1000000000,step=0.001,description='$Qt_{min}$',disabled=False)
Ibmin = BoundedFloatText(value=0.4,min=0.0001,max=1,step=0.0001,description='$b_{min}$',disabled=False)
Idmin = BoundedFloatText(value=87.29741,min=0,max=99.99999,step=0.0001,description='$De_{min}$',disabled=False)

Iqi50 = BoundedFloatText(value=16935,min=0,max=1000000000,step=0.001,description='$Qi_{fit}$',disabled=False)
Iqf50 = BoundedFloatText(value=100,min=0,max=1000000000,step=0.001,description='$Qt_{fit}$',disabled=False)
Ib50 = BoundedFloatText(value=0.5,min=0.0001,max=1,step=0.0001,description='$b_{fit}$',disabled=False)
Id50 = BoundedFloatText(value=81.3626,min=0,max=99.99999,step=0.0001,description='$De_{fit}$',disabled=False)

A=Dropdown(options=['Oil', 'Gas'],value='Oil',description='HC Type:',disabled=False)

In [8]:
interact (tdca,HC=A,qimax=Iqimax,qfmax=Iqfmax,
          bmax=Ibmax,dmax=Idmax,qimin=Iqimin,
          qfmin=Iqfmin,bmin=Ibmin,dmin=Idmin,
          qi50=Iqi50,qf50=Iqf50,b50=Ib50,d50=Id50 );

interactive(children=(Dropdown(description='HC Type:', options=('Oil', 'Gas'), value='Oil'), BoundedFloatText(…

<br />
<br />
<br />
<br />
<br />

Designed by **Fahim N. Is.** , *BS. Petroleum Engineering, The University of Texas at Austin*